<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#DecisionTreeClassifier" data-toc-modified-id="DecisionTreeClassifier-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>DecisionTreeClassifier</a></span></li><li><span><a href="#SGDClassifier" data-toc-modified-id="SGDClassifier-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>SGDClassifier</a></span></li><li><span><a href="#CatBoostClassifier" data-toc-modified-id="CatBoostClassifier-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>CatBoostClassifier</a></span></li><li><span><a href="#RidgeClassifier" data-toc-modified-id="RidgeClassifier-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>RidgeClassifier</a></span></li><li><span><a href="#Тестововые-данные" data-toc-modified-id="Тестововые-данные-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Тестововые данные</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import requests

from tqdm import tqdm
tqdm.pandas()

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
import  numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer 
from sklearn.linear_model import LogisticRegression, SGDClassifier,RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import RandomizedSearchCV
import warnings
import re 
from nltk import pos_tag
import spacy
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.dummy import  DummyClassifier
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer

warnings.filterwarnings('ignore')
RANDOM_STATE = 42

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Serg\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
try:
     data = pd.read_csv('C:/Users/Serg/practicum/database/toxic_comments.csv')
except:    
    data = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
# удалим ненужный признак
data = data.drop(['Unnamed: 0'],axis=1)


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [6]:
# поиск пропусков
data.isna().sum()

text     0
toxic    0
dtype: int64

In [7]:
# поиск дубликатов
data.duplicated().sum()

0

In [8]:
# дисбаланс классов для более быстрой работы SVC and catboost
data.toxic.value_counts(normalize=True)

toxic
0    0.898388
1    0.101612
Name: proportion, dtype: float64

In [9]:
#data = data.sample(10000).reset_index(drop=True)
#data.toxic.value_counts(normalize=True)

In [10]:
# функция очистки текста
def clear_text(text):
    text=text.lower()
    ct = re.sub(r'[^a-zA-Z]', ' ', text) 
    ct=ct.split()
    return " ".join(ct)    




In [11]:
disabled_pipes = [ "parser",  "ner"]
nlp = spacy.load('en_core_web_sm', disable=disabled_pipes)
def lemmatize(doc): 
    sent=[]
    words = []
    for token in doc:
            words.append(token.lemma_)
    return ' '.join(words)

In [12]:
data['text'] = data['text'].apply(clear_text)

In [13]:
#data = data.iloc[:1000]

In [14]:
%%time
data['text']  = list(nlp.pipe(data['text'], n_process=5, batch_size=1000))

CPU times: total: 3min 55s
Wall time: 4min 40s


In [16]:
%%time
# проводим лемматизацию
#data['text'] = data['text'].progress_apply(lemmatizer)
data['text'] = data['text'].progress_apply(lemmatize)

100%|███████████████████████████████████████████████████████████████████████| 159292/159292 [00:04<00:00, 35781.41it/s]

CPU times: total: 4.45 s
Wall time: 4.46 s


In [17]:
data.head()

,text,toxic
0,explanation why the edit make under my usernam...,0
1,d aww he match this background colour I m seem...,0
2,hey man I m really not try to edit war it s ju...,0
3,more I can t make any real suggestion on impro...,0
4,you sir be my hero any chance you remember wha...,0


In [18]:
# разбиваем данные на тренировачную и тестоваю выборки
X = data.drop(['toxic'],axis=1)
y = data['toxic']
X_train,X_test, y_train, y_test = train_test_split(X,y, test_size=0.1,random_state=RANDOM_STATE)


Данные загружены, проведена очистка и лемматизация текста. Обнаружен дисбаланс классов.
Данные подготовлены для обучения

## Обучение

In [21]:
def RSCV(estimator, parametrs):
    return RandomizedSearchCV(
    estimator, 
    parametrs, 
    cv=5,
    scoring='f1',
    error_score='raise',
    n_jobs=-1
)

###  LogisticRegression

In [22]:

preprocessor = ColumnTransformer(
    [
    ('tf-idf', TfidfVectorizer(stop_words='english'),'text')
    ], 
    remainder='passthrough') 

In [23]:
pip_lr = Pipeline(

    [('preprocessor', preprocessor), 
    ('model', LogisticRegression(class_weight='balanced', random_state=RANDOM_STATE))])

param =  {'model__solver': ('liblinear', 'saga','newton-cg', 'lbfgs'),
          'model__C': [0.1,0.5, 1,3, 6,7, 10]}

In [24]:
random_lr = RSCV(pip_lr,param)
random_lr.fit(X_train, y_train)

model_lr = random_lr.best_estimator_

print ('Метрика f1 на тренировочной выборке:', random_lr.best_score_)

Метрика f1 на тренировочной выборке: 0.7641481626696267


### DecisionTreeClassifier

In [25]:
param_dtc =[{
        #'models__max_depth': range(5,10),
        'models__min_samples_leaf': range(5,10),
       # 'models__max_features': range(3,10),
        #'models__min_samples_split': range(3,7)
}]
pipe_dtc = Pipeline(
    [('preprocessor', preprocessor),
    ('models', DecisionTreeClassifier(random_state=RANDOM_STATE))])

In [26]:
random_dtc =  RSCV(pipe_dtc,param_dtc)
random_dtc.fit(X_train, y_train)

model_dtc = random_dtc.best_estimator_

print ('Метрика f1 на тренировочной выборке:', random_dtc.best_score_)

Метрика f1 на тренировочной выборке: 0.7300854924017678


### SGDClassifier

In [27]:
svc = pipe_dtc = Pipeline(
    [('preprocessor', preprocessor),
    ('models', SGDClassifier(class_weight='balanced',random_state=RANDOM_STATE))])
param_svc ={
        
        'models__loss':['hinge', 'log_loss', 'modified_huber', 'squared_hinge'],
        'models__eta0':[0.01,0.05,1,],
        'models__learning_rate':['constant','optimal','invscaling']
        }

In [28]:
random_svc =  RSCV(svc,param_svc)

random_svc.fit(X_train, y_train)

model_svc = random_svc.best_estimator_

print ('Метрика f1 на тренировочной выборке:', random_svc.best_score_)

Метрика f1 на тренировочной выборке: 0.7546342688893455


### CatBoostClassifier

In [29]:
param_cat =[{
     #'models__max_depth': range(10,16)
}]
pip_cat = Pipeline(
    [#('preprocessor', preprocessor),
    ('models',CatBoostClassifier(text_features=['text'],
                                 auto_class_weights='Balanced',
                                 verbose=10,
                                 random_state=RANDOM_STATE))])

In [30]:
random_cat =  RSCV(pip_cat,param_cat)
random_cat.fit(X_train, y_train)
model_cat = random_cat.best_estimator_

print ('Метрика f1 на тренировочной выборке:', random_cat.best_score_)

Learning rate set to 0.085848
0:	learn: 0.6235343	total: 245ms	remaining: 4m 5s
10:	learn: 0.3605215	total: 1.61s	remaining: 2m 24s
20:	learn: 0.3065925	total: 3.19s	remaining: 2m 28s
30:	learn: 0.2858658	total: 4.77s	remaining: 2m 29s
40:	learn: 0.2764345	total: 6.17s	remaining: 2m 24s
50:	learn: 0.2695587	total: 7.6s	remaining: 2m 21s
60:	learn: 0.2640811	total: 8.91s	remaining: 2m 17s
70:	learn: 0.2603837	total: 10.2s	remaining: 2m 13s
80:	learn: 0.2575311	total: 11.4s	remaining: 2m 9s
90:	learn: 0.2543325	total: 12.7s	remaining: 2m 7s
100:	learn: 0.2509096	total: 14.2s	remaining: 2m 6s
110:	learn: 0.2485753	total: 15.9s	remaining: 2m 7s
120:	learn: 0.2458992	total: 17.1s	remaining: 2m 4s
130:	learn: 0.2434828	total: 18.3s	remaining: 2m 1s
140:	learn: 0.2408988	total: 19.5s	remaining: 1m 58s
150:	learn: 0.2379280	total: 20.8s	remaining: 1m 56s
160:	learn: 0.2351421	total: 22.4s	remaining: 1m 56s
170:	learn: 0.2323549	total: 23.7s	remaining: 1m 54s
180:	learn: 0.2297809	total: 25s	re

### RidgeClassifier

In [31]:
param_rc =[{
        'models__alpha': [1,10,100]
}]
pip_rc = Pipeline(
    [('preprocessor', preprocessor),
    ('models',RidgeClassifier(class_weight='balanced',
                                 random_state=RANDOM_STATE))])

In [32]:
random_rc =  RSCV(pip_rc,param_rc)
random_rc.fit(X_train, y_train)
model_rc = random_rc.best_estimator_

print ('Метрика f1 на тренировочной выборке:', random_rc.best_score_)

Метрика f1 на тренировочной выборке: 0.708176295763984


### Тестововые данные

Итак лучшая модель оказалась Логическая регресия f1=0.77 её и будем использовать на тестовой выборке.

In [33]:
test_pred = model_lr.predict(X_test)
print ('Метрика f1 на тестовой выборке:', f1_score(y_test,test_pred))

Метрика f1 на тестовой выборке: 0.7777777777777779


Условия задачи выполняются f1 >0.75

Проверим на адеватность

In [34]:
dc =DummyClassifier(strategy='uniform')
dc.fit(X_train, y_train)
f1_score(y_test,dc.predict(X_test))

0.16825363607826724

Проверка пройдена

## Выводы

Были загружены и исследованы  данные
- удален лишний признак.
- Проведена очистка и лемматизация текстов.

Были обучены пять моделей, метрика f1:
- CatBoostClassifier, f1-0.73.
- SGDClassifier, f1- 0.75.
- DecisionTreeClassifier, f1 -0.73.
- LogisticRegression, f1 -0.77.
- RidgeClassifier, f1 - 0.71

Определена лучшая модель - LogisticRegression
- проверка на адекватность пройдена.
- на тестовых данных метрика f1 = 0.77.